# Install Libraries
Install information can be found [here](https://satijalab.org/seurat/articles/install.html)\
Vignette [here](https://satijalab.org/seurat/articles/integration_mapping.html)

In [1]:
#Seurat parameters
reference_data = "path-to-seurat-object" # Seurat object for reference data
query_data = "path-to-seurat-object" # Seurat object for query data

genome = "genome-name" # either hg38 or mm10

normalization_method = "LogNormalize"
normalization_scale_factor = 10000

variable_features_method = "vst"
variable_features_num = 2000

weight.reduction = "pca" # Dimensional reduction to use for the weighting anchors.
n_dims = 30 # Set of dimensions to use in the anchor weighting procedure. If NULL, the same dimensions that were used to find anchors will be used for weighting.

threads = 8
prefix = "prefix" #project name

#Terra specific parameters
table_name = "demux_BH3KTLDMXY"
experiment_name = "gm12878_fresh_RNA"

#Papermill specific parameters
papermill = TRUE

#jupyter notebook plot sizes
options(repr.plot.width=20, repr.plot.height=15)

In [2]:
#########################
# For test
reference_data = "../../../ReferenceData/BrainAgingSpatialAtlas_snRNAseq.rds"
query_data = "../../../QueryData/MouseBrain/SS-PKR-129-192-PLATE1-LEFT-HALF.rna.seurat.filtered_rds.mm10.rds"
genome = "mm10"

In [3]:
papermill <- as.logical(papermill)

In [4]:
if (!requireNamespace("Seurat", quietly = TRUE))
    install.packages("Seurat")
if (!requireNamespace("future", quietly = TRUE))
    install.packages("future")
if (!requireNamespace("logr", quietly = TRUE))
    install.packages("logr")
if (!requireNamespace("grid", quietly = TRUE))
    install.packages("grid")
if (!requireNamespace("dplyr", quietly = TRUE))
    install.packages("dplyr")
if (!requireNamespace("gridExtra", quietly = TRUE))
    install.packages("gridExtra")
if (!requireNamespace("ggplot2", quietly = TRUE))
    install.packages("ggplot2")
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
if (!requireNamespace("EnsDb.Mmusculus.v79", quietly = TRUE))
    BiocManager::install("EnsDb.Mmusculus.v79")
if (!requireNamespace("EnsDb.Hsapiens.v86", quietly = TRUE))
    BiocManager::install("EnsDb.Hsapiens.v86")

suppressMessages(library(Seurat))
suppressMessages(library(future))
suppressMessages(library(logr))
suppressMessages(library(dplyr))
suppressMessages(library(grid))
suppressMessages(library(gridExtra))
suppressMessages(library(ggplot2))
suppressMessages(library(EnsDb.Mmusculus.v79))
suppressMessages(library(EnsDb.Hsapiens.v86))

future.seed=TRUE
plan("multisession", workers = threads)
options("logr.notes" = FALSE)
options(future.globals.maxSize=10e9)
set.seed(1234)

In [5]:
# Function to convert gene ID to symbol
create_seurat_obj_with_gene_symbol <- function(obj, genome){

    # get gene symbol
    if(genome == "hg38"){
        gene.id <- ensembldb::select(EnsDb.Hsapiens.v86, 
                                      keys= rownames(obj), 
                                      keytype = "GENEID", 
                                      columns = c("SYMBOL","GENEID"))
    
    } else if(genome == "mm10"){
        gene.id <- ensembldb::select(EnsDb.Mmusculus.v79, 
                                  keys= rownames(obj), 
                                  keytype = "GENEID", 
                                  columns = c("SYMBOL","GENEID"))
    }
    # remove genes with empty symbol
    gene.id <- subset(gene.id, gene.id$SYMBOL != "")

    # make gene symbol unique
    gene.id$Unique_SYMBOL <- make.unique(gene.id$SYMBOL, "")
    
    counts <- obj@assays$RNA@counts
    counts <- counts[gene.id$GENEID, ]
    rownames(counts) <- gene.id$Unique_SYMBOL

    obj2 <- CreateSeuratObject(counts = counts, meta.data = obj@meta.data)
    
    return(obj2)
}

In [6]:
# #Function to save plots
# plot_filename = paste0(prefix,".rna.seurat.annotation.plots.",genome)
# dir.create(plot_filename, showWarnings=F)
# printPNG <- function(name, plotObject, papermill, wf=22, hf=11){
#     filename = paste0(plot_filename,"/",prefix,".rna.seurat.",name,".",genome,".png")
#     if(papermill){
#         ggsave(plot = plotObject, filename = filename, width = wf, height = hf)
#     }
# }

# #Function to create plots
# create_plot = function(plot_list, title, subtitle, heights=unit(c(1,1,8), rep("in",3)), width = 7){
#     g = c(list(title),list(subtitle),plot_list)
#     N = length(plot_list)
#     laym = rbind(rep(1,N),rep(2,N),(3:(N+2)))
#     widths = unit(rep(width, length(plot_list)), rep("in",length(plot_list)))
#     obj = arrangeGrob(grobs=g,layout_matrix=laym, heights=heights, widths = widths)
#     return(obj)
# }


#Create log file
logfile <- file.path(paste0(prefix,".rna.seurat.annotation.logfile.", genome, ".txt"))
lf <- log_open(logfile)

In [7]:
# Read reference data
tryCatch(
    {
        log_print("# Reading reference data...")
        obj.ref <- readRDS(reference_data)
        log_print("SUCCESSFUL: Reading reference data")
    
    },
    error = function(cond) {
        log_print("ERROR: Reading reference data")
        log_print(cond)
    }
)

[1] "# Reading reference data..."
[1] "SUCCESSFUL: Reading reference data"


In [8]:
# Read query data
tryCatch(
    {
        log_print("# Reading query data...")
        obj.query <- readRDS(query_data)
        log_print("SUCCESSFUL: Reading query data")
    
    },
    error = function(cond) {
        log_print("ERROR: Reading query data")
        log_print(cond)
    }
)

[1] "# Reading query data..."
[1] "SUCCESSFUL: Reading query data"


In [9]:
# Convert gene ID to symbol for reference data
tryCatch(
    {
        log_print("# Converting gene id to symbol for reference data")
        obj.ref <- create_seurat_obj_with_gene_symbol(obj = obj.ref, 
                                                      genome = genome)
        log_print("SUCCESSFUL: Converting gene id to symbol for reference data")

    },
    error = function(cond) {
        log_print("ERROR: Converting gene id to symbol for reference data")
        log_print(cond)
    }
)

[1] "# Converting gene id to symbol for reference data"
[1] "SUCCESSFUL: Converting gene id to symbol for reference data"


In [10]:
# Subset reference data
tryCatch(
    {
        log_print("# Subseting reference data")
        gene.common <- intersect(rownames(obj.ref), rownames(obj.query))
        
        log_print("SUCCESSFUL: Subseting reference data")
        obj.ref <-  subset(obj.ref, features = gene.common)
        obj.query <-  subset(obj.query, features = gene.common)
    },
    warning = function(conda){
        log_print("WARNING: Subseting reference data")
        log_print(cond)
    }
     error = function(cond) {
        log_print("ERROR: Subseting reference data")
        log_print(cond)
    }
    
)

[1] "# Subseting reference data"
[1] "SUCCESSFUL: Subseting reference data"


Warning message:
"Not all features provided are in this Assay object, removing the following feature(s): Tafa1, Gm32338, BC005561, Nrg1, Gm32647, Gm42418, Gm10754, Tafa2, Gm10419, Gm26871, mt-Co1, 2010300C02Rik, Gmnc, 2700081O15Rik, 4921539H07Rik, Gm33228, Gm20754, Adgrl4, Rflnb, Lhfpl3, March1, Tmem94, Gm10649, mt-Nd1, Gm5127, Epb41l4a, Drd1, Gm28905, Mrm2, Pakap.1, Gm5820, Rmst, C78859, Gm14051, Gm10516, C230072F16Rik, Gm30371, C130073E24Rik, Gm39185, Gm44593, Twnk, Rtl4, Gm867, Gm38413, Gm12367, Gm49969, Gm45356, Gm6260, BC051408, Gm20457, Gm31645, Gm13561, Ints11, Gm14412, Gm11906, Rps6ka2, Dele1, Gm49678, Tmem131l, Gm20125, Gm45459, Gm49164, Sdhaf4, Gm11542, Adgrl2, Gm1604a, Usf3, Ccn2, Gm42196, Gm15587, Gm4890, Phf24, Gm43598, Gm42851, P3h1, Gm30382, Gm27188, Gm26673, Mrvi1, Vxn, Gsdme, Gm33677, Cip2a, Gm20387, 4930547E14Rik, Znrd2, C530008M17Rik, Gm2164, Gm14033, Gm39043, Plpp3, Cramp1l, Gm12689, Gm26621, Gm38560, 4930488L21Rik, Fam71d, 5330417C22Rik, 4933406B17Rik, Gm19325, 953

In [ ]:
# Predict labels for query dataset
tryCatch(
    {
        log_print("# Predicting labels for query data")
        
        obj.ref <- obj.ref %>%
            NormalizeData(verbose = FALSE) %>%
            FindVariableFeatures(selection.method = variable_features_method, 
                                 nfeatures = variable_features_num)
        
        obj.query <- obj.query %>%
            NormalizeData(verbose = FALSE) %>%
            FindVariableFeatures(selection.method = variable_features_method,
                                 nfeatures = variable_features_num)
        
        transfer.anchors <- FindTransferAnchors(
            reference = obj.ref,
            query = obj.query,
            reduction = "cca",
            verbose = FALSE
        )
        
        predictions <- TransferData(anchorset = transfer.anchors, 
                                    refdata = obj.ref$cell_type,
                                    weight.reduction = obj.query[[weight.reduction]],
                                    dims = 1:n_dims,
                                   verbose = FALSE)
        
        log_print("SUCCESSFUL: Predicting labels for query data")
    },
     error = function(cond) {
        log_print("ERROR: Predicting labels for query data")
        log_print(cond)
    }
    
)

In [ ]:
# get_file <- function(path){
#     dest <- getwd()
#     gsutil_cp(path, dest)
#     name <- basename(path)
#     return(name)
# }

# if (!papermill){
#     table <- avtable(table_name)
#     rna_matrix <- get_file(table$h5_matrix[table[, sprintf('%s_id', table_name)] == experiment_name])
# }

In [ ]:
# #Pre-filtered violin QC plots 

# tryCatch({
#         log_print("# Pre-filtered Violin Plot")
    
#         #Code start to create pre-filtered rna QC violin plot
    
#         obj <- VlnPlot(rna, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"), ncol = 3, combine = FALSE, pt.size = 0)
                
#         tg <- textGrob('Pre-filtered RNA QC plots', gp = gpar(fontsize = 25, fontface = 'bold', col = 'red'))
#         sg <- textGrob(paste0("Library: ", prefix, "\n", 
#                         nrow(x = rna), " genes across ", ncol(x = rna), 
#                         " barcodes \n------------------------------------ \n"), 
#                         gp = gpar(fontsize = 18, fontface = 'bold'))

#         plot_list = list(obj[[1]] + scale_y_log10() + NoLegend() + theme(axis.title=element_text(size=14), axis.text=element_text(size=10), legend.title=element_text(size=14), legend.text=element_text(size=8.5)), 
#                          obj[[2]] + scale_y_log10() + NoLegend() + theme(axis.title=element_text(size=14), axis.text=element_text(size=10), legend.title=element_text(size=14), legend.text=element_text(size=8.5)), 
#                          obj[[3]] + theme(axis.title=element_text(size=14), axis.text=element_text(size=10), legend.title=element_text(size=14), legend.text=element_text(size=8.5)))
        
#         obj = create_plot(plot_list, tg, sg)
#         grid.draw(obj)
#         printPNG(name = "prefiltered_violin", plotObject = obj, papermill = papermill)
    
#         #Code end to create violin plot
    
#         log_print("SUCCESSFUL: Pre-filtered Violin Plot")
#     },
#     error = function(cond) {
#         log_print("ERROR: Pre-filtered Violin Plot")
#         log_print(cond)
#     }
# )

In [ ]:
# # Create feature plots

# tryCatch({
#         log_print("# Pre-filtered QC Scatterplots")
    
#         #Code start to create feature plots
    
#         plot1 <- FeatureScatter(rna, feature1 = "nCount_RNA", feature2 = "percent.mt") + 
#             theme(axis.title=element_text(size=14), axis.text=element_text(size=10), legend.title=element_text(size=14), legend.text=element_text(size=10))
#         plot2 <- FeatureScatter(rna, feature1 = "nCount_RNA", feature2 = "nFeature_RNA") +
#             theme(axis.title=element_text(size=14), axis.text=element_text(size=10), legend.title=element_text(size=14), legend.text=element_text(size=10))
        
#         tg <- textGrob('Pre-filtered QC scatterplots', gp = gpar(fontsize = 25, fontface = 'bold', col = 'red'))
#         sg <- textGrob(paste0("Library: ", prefix, "\n", 
#                         nrow(x = rna), " genes across ", ncol(x = rna), 
#                         " barcodes \n------------------------------------ \n"), 
#                         gp = gpar(fontsize = 18, fontface = 'bold'))

#         plot_list = list(plot1,plot2)
#         obj = create_plot(plot_list, tg, sg ,heights = unit(c(1,1,8), rep("in",3)), width = 10)
#         grid.draw(obj)
#         printPNG(name = "prefiltered_qc_scatterplots", plotObject = obj, papermill = papermill)
    
#         #Code end to create feature plots
    
#         log_print("SUCCESSFUL: Pre-filtered QC Scatterplots")
#     },
#     error = function(cond) {
#         log_print("ERROR: Pre-filtered QC Scatterplots")
#         log_print(cond)
#     }
# )

In [ ]:
# #Subset Seurat Object by QC filters

# rna = tryCatch({
#         log_print("# Subset Seurat Object")
    
#         #Code start to create seurat object
    
#         #rna <- CreateSeuratObject(counts = data, project = prefix, min.cells = min_cells, min.features = min_features)
#         #rna[["percent.mt"]] <- PercentageFeatureSet(rna, pattern = "^MT-")
        
#         rna <- subset(rna, subset = PassQC == TRUE)
    
#         #write postfiltered .h5 matrix  
#         write_dgCMatrix_h5(GetAssayData(object = rna, slot = "counts"), 
#                            cols_are = "barcodes", 
#                            h5_target = paste0(prefix,".rna.seurat.filtered_matrix.",genome,".h5"), 
#                            ref_name = prefix)
    
#         #Code end to create seurat object
    
#         log_print("SUCCESSFUL: Subset Seurat Object")
#         return(rna)
#     },
#     error = function(cond) {
#         log_print("ERROR: Subset Seurat Object")
#         log_print(cond)
#     }
# )

In [ ]:
# #Post-filtered violin QC plots 

# tryCatch({
#         log_print("# Post-filtered Violin Plot")
    
#         #Code start to create post-filtered rna QC violin plot
        
#         obj <- VlnPlot(rna, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"), ncol = 3, combine = F, pt.size = 0)
#         tg <- textGrob('Post-filtered RNA QC plots', gp = gpar(fontsize = 25, fontface = 'bold', col = 'red'))
#         sg <- textGrob(paste0("Library: ", prefix, "\n", 
#                         nrow(x = rna), " genes across ", ncol(x = rna), 
#                         " barcodes \n------------------------------------ \n"), 
#                         gp = gpar(fontsize = 18, fontface = 'bold'))

#         plot_list = list(obj[[1]] + scale_y_log10() + NoLegend() + theme(axis.title=element_text(size=14), axis.text=element_text(size=10), legend.title=element_text(size=14), legend.text=element_text(size=10)), 
#                          obj[[2]] + scale_y_log10() + NoLegend() + theme(axis.title=element_text(size=14), axis.text=element_text(size=10), legend.title=element_text(size=14), legend.text=element_text(size=10)), 
#                          obj[[3]] + theme(axis.title=element_text(size=14), axis.text=element_text(size=10), legend.title=element_text(size=14), legend.text=element_text(size=10)))
        
#         obj = create_plot(plot_list, tg, sg)
#         grid.draw(obj)
#         printPNG(name = "postfiltered_violin", plotObject = obj, papermill = papermill)
        
#         #Code end to create violin plot
    
#         log_print("SUCCESSFUL: Post-filtered Violin Plot")
#     },
#     error = function(cond) {
#         log_print("ERROR: Post-filtered Violin Plot")
#         log_print(cond)
#     }
# )

In [ ]:
# # Create feature plots

# tryCatch({
#         log_print("# Post-filtered QC Scatterplots")
    
#         #Code start to create feature plots
    
#         plot1 <- FeatureScatter(rna, feature1 = "nCount_RNA", feature2 = "percent.mt") +
#             theme(axis.title=element_text(size=14), axis.text=element_text(size=10), legend.title=element_text(size=14), legend.text=element_text(size=10))
#         plot2 <- FeatureScatter(rna, feature1 = "nCount_RNA", feature2 = "nFeature_RNA") +
#             theme(axis.title=element_text(size=14), axis.text=element_text(size=10), legend.title=element_text(size=14), legend.text=element_text(size=10))
        
#         tg <- textGrob('Post-filtered QC scatterplots', gp = gpar(fontsize = 25, fontface = 'bold', col = 'red'))
#         sg <- textGrob(paste0("Library: ", prefix, "\n", 
#                         nrow(x = rna), " genes across ", ncol(x = rna), 
#                         " barcodes \n------------------------------------ \n"), 
#                         gp = gpar(fontsize = 18, fontface = 'bold'))

#         plot_list = list(plot1,plot2)
#         obj = create_plot(plot_list, tg, sg, heights = unit(c(1,1,8), rep("in",3)), width = 10)
#         grid.draw(obj)
#         printPNG(name = "postfiltered_qc_scatterplots", plotObject = obj, papermill = papermill)
        
#         #Code end to create feature plots
    
#         log_print("SUCCESSFUL: Post-filtered QC Scatterplots")
#     },
#     error = function(cond) {
#         log_print("ERROR: Post-filtered QC Scatterplots")
#         log_print(cond)
#     }
# )

In [ ]:
# #Set Seurat obj to NULL if number of barcodes exceed 300k - this is a temp fix until better alternative

# rna = tryCatch({
#     log_print("# Check number of barcodes")
    
#         if(nrow(rna@meta.data) > 250000){
#             stop("Number of barcodes exceeding 250k; terminating")
#         } else{
#             log_print("SUCCESSFUL: Check number of barcodes")
#         }
#         return(rna)
#     },
#     error = function(cond) {
#         log_print("ERROR: Check number of barcodes")
#         log_print(cond)
#         rna = NULL
#         return(rna)
#     }
# )

In [ ]:
# # Normalization

# tryCatch({
#         log_print("Normalization")
    
#         #Code start to normalize
    
#         rna <- NormalizeData(rna, normalization.method = normalization_method, scale.factor = normalization_scale_factor)
    
#         #Code end to normalize
    
#         log_print("SUCCESSFUL: Normalization")
#     },
#     error = function(cond) {
#         log_print("ERROR: Normalization")
#         log_print(cond)
#     }
# )


In [ ]:
# # Find Variable Features

# tryCatch({
#         log_print("Finding Variable Features")
        
#         #Code start to find variable features
    
#         rna <- FindVariableFeatures(rna, selection.method = variable_features_method, nfeatures = variable_features_num)
        
#         # Identify the 10 most highly variable genes
#         top10 <- head(VariableFeatures(rna), 10)

#         # plot variable features with and without labels
#         plot1 <- VariableFeaturePlot(rna)
#         plot2 <- LabelPoints(plot = plot1, points = top10, repel = TRUE)
    
#         tg <- textGrob('Variable Genes: Mean Expression-by-Variance Scatterplot', gp = gpar(fontsize = 25, fontface = 'bold', col = 'red'))
#         sg <- textGrob(paste0("Library: ", prefix, "\n", 
#                         nrow(x = rna), " genes across ", ncol(x = rna), 
#                         " barcodes \n------------------------------------ \n"), 
#                         gp = gpar(fontsize = 18, fontface = 'bold'))

#         plot_list = list(plot2)
#         obj = create_plot(plot_list, tg, sg, width = 12, heights = unit(c(1,1,8), rep("in",3)))
#         grid.draw(obj)
#         printPNG(name = "variable_genes", plotObject = obj, papermill = papermill, wf = 14, hf = 12)
        
#         log_print("SUCCESSFUL: Finding Variable Features")
#     },
#     error = function(cond) {
#         log_print("ERROR: Finding Variable Features")
#         log_print(cond)
#     }
# )

In [ ]:
# # Scaling

# tryCatch({
#         log_print("Scaling")
    
#         #Code start to scale
    
#         all.genes <- rownames(rna)
#         rna <- ScaleData(rna, features = all.genes)
    
#         #Code end to scale
    
#         log_print("SUCCESSFUL: Scaling")
#     },
#     error = function(cond) {
#         log_print("ERROR: Scaling")
#         log_print(cond)
#     }
# )


In [ ]:
# #Principal Component Analysis

# tryCatch({
#         log_print("Principal Component Analysis")

#         # Code start to run PCA    
    
#         rna <- RunPCA(rna, features = VariableFeatures(object = rna))
#         obj <- VizDimLoadings(rna, dims = 1:dim_loadings_dim, reduction = "pca")
        
#         tg <- textGrob('PCA: Dimension Loadings', gp = gpar(fontsize = 25, fontface = 'bold', col = 'red'))
#         sg <- textGrob(paste0("Library: ", prefix, "\n", 
#                         nrow(x = rna), " genes across ", ncol(x = rna), 
#                         " barcodes \n------------------------------------ \n"), 
#                         gp = gpar(fontsize = 18, fontface = 'bold'))
#         plot_list = list(obj[[1]], obj[[2]])
#         obj = create_plot(plot_list, tg, sg, width = 10, heights = unit(c(1,1,8), rep("in",3)))
#         grid.draw(obj)
#         printPNG(name = "pca_dim_loadings", plotObject = obj, papermill = papermill)
            
#         # Code end to run PCA  

#         log_print("SUCCESSFUL: Principal Component Analysis")
#     },
#     error = function(cond) {
#         log_print("ERROR: Principal Component Analysis")
#         log_print(cond)
#     }
# )

In [ ]:
# # PCA plot

# tryCatch({
#         log_print("PCA plot")

#         # Code start to create PCA plot 
    
#         obj <- DimPlot(rna, reduction = "pca")
    
#         tg <- textGrob('PCA Plot', gp = gpar(fontsize = 25, fontface = 'bold', col = 'red'))
#         sg <- textGrob(paste0("Library: ", prefix, "\n", 
#                         nrow(x = rna), " genes across ", ncol(x = rna), 
#                         " barcodes \n------------------------------------ \n"), 
#                         gp = gpar(fontsize = 18, fontface = 'bold'))
#         plot_list = list(obj[[1]])
#         obj = create_plot(plot_list, tg, sg, width = 12, heights = unit(c(1,1,8), rep("in",3)))
#         grid.draw(obj)
#         printPNG(name = "pca", plotObject = obj, papermill = papermill, wf = 14, hf = 12)
        
#         # Code end to create PCA plot  

#         log_print("SUCCESSFUL: PCA plot")
#     },
#     error = function(cond) {
#         log_print("ERROR: PCA plot")
#         log_print(cond)
#     }
# )

In [ ]:
# # Heatmap

# tryCatch({
#         log_print("Heatmap")

#         # Code start to create heatmap 
    
#         obj <- DimHeatmap(rna, dims = heatmap_dim, cells = heatmap_cells, fast = FALSE, balanced = heatmap_balanced)
#         tg <- textGrob('Heatmap', gp = gpar(fontsize = 25, fontface = 'bold', col = 'red'))
#         sg <- textGrob(paste0("Library: ", prefix, "\n", 
#                         nrow(x = rna), " genes across ", ncol(x = rna), 
#                         " barcodes \n------------------------------------ \n"), 
#                         gp = gpar(fontsize = 18, fontface = 'bold'))
#         plot_list = list(obj[[1]])
#         obj = create_plot(plot_list, tg, sg, width = 12, heights = unit(c(1,1,8), rep("in",3)))
#         grid.draw(obj)
#         printPNG(name = "heatmap", plotObject = obj, papermill = papermill, wf = 14, hf = 12)
        
#         # Code end to create heatmap 

#         log_print("SUCCESSFUL: Heatmap")
#     },
#     error = function(cond) {
#         log_print("ERROR: Heatmap")
#         log_print(cond)
#     }
# )

In [ ]:
# # Jackstraw Plot

# tryCatch({
#         log_print("Jackstraw Plot")

#         # Code start to create jackstraw plot 
    
#         rna <- JackStraw(rna, num.replicate = jackstraw_replicates)
#         rna <- ScoreJackStraw(rna, dims = 1:jackstraw_score_dim)
#         obj <- JackStrawPlot(rna, dims = 1:jackstraw_plot_dim)
    
#         tg <- textGrob('Jackstraw Plot', gp = gpar(fontsize = 25, fontface = 'bold', col = 'red'))
#         sg <- textGrob(paste0("Library: ", prefix, "\n", 
#                         nrow(x = rna), " genes across ", ncol(x = rna), 
#                         " barcodes \n------------------------------------ \n"), 
#                         gp = gpar(fontsize = 18, fontface = 'bold'))
#         plot_list = list(obj)
#         obj = create_plot(plot_list, tg, sg, width = 10,heights = unit(c(1,1,8), rep("in",3)))
#         grid.draw(obj)
#         printPNG(name = "jackstraw", plotObject = obj, papermill = papermill)
            
#         # Code end to create jackstraw plot 

#         log_print("SUCCESSFUL: Jackstraw Plot")
#     },
#     error = function(cond) {
#         log_print("ERROR: Jackstraw Plot")
#         log_print(cond)
#     }
# )

In [ ]:
# # Elbow Plot

# tryCatch({
#         log_print("Elbow Plot")

#         # Code start to create elbow plot 
    
#         obj <- ElbowPlot(rna)
#         tg <- textGrob('Elbow Plot', gp = gpar(fontsize = 25, fontface = 'bold', col = 'red'))
#         sg <- textGrob(paste0("Library: ", prefix, "\n", 
#                         nrow(x = rna), " genes across ", ncol(x = rna), 
#                         " barcodes \n------------------------------------ \n"), 
#                         gp = gpar(fontsize = 18, fontface = 'bold'))
#         plot_list = list(obj)
#         obj = create_plot(plot_list, tg, sg, width = 10,heights = unit(c(1,1,8), rep("in",3)))
#         grid.draw(obj)
#         printPNG(name = "elbow", plotObject = obj, papermill = papermill)
                
#         # Code end to create elbow plot 

#         log_print("SUCCESSFUL: Elbow Plot")
#     },
#     error = function(cond) {
#         log_print("ERROR: Elbow Plot")
#         log_print(cond)
#     }
# )

In [ ]:
# # Run UMAP

# tryCatch({
#         log_print("Run UMAP")

#         # Code start to run umap 
    
#         rna <- FindNeighbors(rna, dims = 1:umap_dim)
#         rna <- FindClusters(rna, resolution = umap_resolution)
#         rna <- RunUMAP(rna, dims = 1:umap_dim)
    
#         # Code end to run umap 

#         log_print("SUCCESSFUL: Run UMAP")
#     },
#     error = function(cond) {
#         log_print("ERROR: Run UMAP")
#         log_print(cond)
#     }
# )

In [ ]:
# # UMAP Plot

# tryCatch({
#         log_print("UMAP Plot")

#         # Code start to create UMAP plot 
    
#         obj <- DimPlot(rna, reduction = "umap") +              
#             theme(axis.title=element_text(size=14), axis.text=element_text(size=10), legend.title=element_text(size=14), legend.text=element_text(size=10))                                                                 

#         tg <- textGrob('UMAP Plot: uwot Clustering', gp = gpar(fontsize = 25, fontface = 'bold', col = 'red'))
#         sg <- textGrob(paste0("Library: ", prefix, "\n", 
#                         nrow(x = rna), " genes across ", ncol(x = rna), 
#                         " barcodes \n------------------------------------ \n"), 
#                         gp = gpar(fontsize = 18, fontface = 'bold'))
#         plot_list = list(obj)
#         obj = create_plot(plot_list, tg, sg, width = 12)
#         grid.draw(obj)
#         printPNG(name = "umap", plotObject = obj, papermill = papermill, wf = 14, hf = 12)
    
#         grid.newpage()
    
#         #obj <- FeaturePlot(object = rna, features = "nCount_RNA", reduction = "umap")
#         df = as.data.frame(rna[["umap"]]@cell.embeddings)
#         df$nCount_RNA = rna$nCount_RNA
    
#         obj <- ggplot(data=df) + geom_point(aes(x = UMAP_1, y = UMAP_2, color = log10(nCount_RNA))) + 
#         scale_color_viridis_b() + 
#         theme_bw() + 
#         theme(plot.title = element_text(hjust = 0.5),panel.border = element_blank(), panel.grid.major = element_blank(), panel.grid.minor = element_blank(), axis.line = element_line(colour = "black"))+              
#         theme(axis.title=element_text(size=14), axis.text=element_text(size=10), legend.title=element_text(size=14), legend.text=element_text(size=10))                                                                 

        
#         tg <- textGrob('UMAP Plot: Color by # of RNA molecules', gp = gpar(fontsize = 25, fontface = 'bold', col = 'red'))
#         sg <- textGrob(paste0("Library: ", prefix, "\n", 
#                         nrow(x = rna), " genes across ", ncol(x = rna), 
#                         " barcodes \n------------------------------------ \n"), 
#                         gp = gpar(fontsize = 18, fontface = 'bold'))
#         plot_list = list(obj)
#         obj = create_plot(plot_list, tg, sg, width = 12)
#         grid.draw(obj)
#         printPNG(name = "umap_rna_count", plotObject = obj, papermill = papermill, wf = 14, hf = 12)
    
#         grid.newpage()
    
#         #obj <- FeaturePlot(object = rna, features = "nFeature_RNA", reduction = "umap") 
#         df$nFeature_RNA = rna$nFeature_RNA
    
#         obj <- ggplot(data=df) + geom_point(aes(x = UMAP_1, y = UMAP_2, color = log10(nFeature_RNA))) + 
#         scale_color_viridis_b() + 
#         theme_bw() + 
#         theme(plot.title = element_text(hjust = 0.5),panel.border = element_blank(), panel.grid.major = element_blank(), panel.grid.minor = element_blank(), axis.line = element_line(colour = "black"))+              
#         theme(axis.title=element_text(size=14), axis.text=element_text(size=10), legend.title=element_text(size=14), legend.text=element_text(size=10))                                                                 

        
#         tg <- textGrob('UMAP Plot: Color by # of genes detected', gp = gpar(fontsize = 25, fontface = 'bold', col = 'red'))
#         sg <- textGrob(paste0("Library: ", prefix, "\n", 
#                         nrow(x = rna), " genes across ", ncol(x = rna), 
#                         " barcodes \n------------------------------------ \n"), 
#                         gp = gpar(fontsize = 18, fontface = 'bold'))
#         plot_list = list(obj)
#         obj = create_plot(plot_list, tg, sg, width = 12)
#         grid.draw(obj)
#         printPNG(name = "umap_gene_count", plotObject = obj, papermill = papermill, wf = 14, hf = 12)
    
#         grid.newpage()
    
#         obj <- FeaturePlot(object = rna, features = "percent.mt", reduction = "umap")
#         tg <- textGrob('UMAP Plot: Color by mitochondria percent', gp = gpar(fontsize = 25, fontface = 'bold', col = 'red'))
#         sg <- textGrob(paste0("Library: ", prefix, "\n", 
#                         nrow(x = rna), " genes across ", ncol(x = rna), 
#                         " barcodes \n------------------------------------ \n"), 
#                         gp = gpar(fontsize = 18, fontface = 'bold'))
#         plot_list = list(obj)
#         obj = create_plot(plot_list, tg, sg, width = 12)
#         grid.draw(obj)
#         printPNG(name = "umap_mito", plotObject = obj, papermill = papermill, wf = 14, hf = 12)
        
#         grid.newpage()
    
#         # Code end to create UMAP plot 

#         log_print("SUCCESSFUL: UMAP Plot")
#     },
#     error = function(cond) {
#         log_print("ERROR: UMAP Plot")
#         log_print(cond)
#     }
# )

In [ ]:
# #Create final output files

# tryCatch({
#         log_print("# Final output files")
    
#         #Code start to create final output files
    
#         files2zip <- dir(plot_filename, full.names = TRUE)
#         zip(zipfile = paste0(plot_filename,".zip"), files = files2zip)

#         saveRDS(rna, file = paste0(prefix,".rna.seurat.filtered_rds.",genome,".rds"))

#         #Code end to create final output files
    
#         log_print("SUCCESSFUL: Final output files")
#     },
#     error = function(cond) {
#         log_print("ERROR: Final output files")
#         log_print(cond)
#     }
# )

# log_close()